In [39]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import sklearn
from datetime import datetime
from sklearn.model_selection import train_test_split as tt
from sklearn.metrics import accuracy_score as acc
%matplotlib inline

In [46]:
file = open("dummy.csv", "r")
file1 = open("data_nn.csv", "w")
file1.write('"Time" "Ping Local" "Ping LocalApp-CorpApp" "Ping LocalApp-CorpDB" "Load Average LocalApp" "Load Average LocalDB" "Load Average CORPApp" "TNS ping LocalApp-LocalDB" "TNS ping LocalApp-CorpDB" "Class0" "Class1" "Class2" "Class3"\n')
hr = 12
m = 00
for line in file:
    l = len(line)
    dat = list(line.split())
    if (dat[1] == 'x' or dat[5] == 'x'):
        line = str(hr)+":"+str(m)+" "
        for da in dat:
            if (da == 'x'):
                da = '9999999999'
            line+=da+" "
        line += "0 1 0 0\n"
    elif (dat[2] == 'x' or dat[7] == 'x'):
        line = str(hr)+":"+str(m)+" "
        for da in dat:
            if (da == 'x'):
                da = '9999999999'
            line+=da+" "
        line += "0 0 1 0\n"
    elif (dat[3] == 'x' or dat[4] == 'x' or dat[6] == 'x'):
        line = str(hr)+":"+str(m)+" "
        for da in dat:         
            if (da == 'x'):
                da = '9999999999'
            line+=da+" "
        line += "0 0 0 1\n"
    else:
        line1 = str(hr)+":"+str(m)+" "
        line=line1+line[:-1] + " 1 0 0 0" + line[-1]
    file1.write(line)
    m=m+1
    if(m==60):
       m=0 
       hr=hr+1
file.close()
file1.close()

In [47]:
x = tf.placeholder(tf.float32,[None,8])
#weights
W = tf.Variable([[0.15463702, -1.21701765, 0.52780521, -1.4738549],
                [0.42833516, -0.04660674, 0.82068664, 0.09615696],
                [0.62039953, 0.93298858, -0.19275554, -1.06650257],
                [-0.05696351, -1.84619224, -0.93848866, -0.38569927],
                [-0.67261547, 0.19050605, -0.29760489, -0.62254256],
                [-1.1083765, -2.41506457, 0.25277814, -2.16972232],
                [0.35110912, 0.16467018, 2.88074422, -0.33394173],
                [1.98250139, -0.09985847, -0.04889834, -0.10028272]])
#W2 = tf.Variable(tf.random_normal([16,4]))
#biases
b = tf.Variable([0.13422392, -0.29879299, 0.13994822, 1.51326454])
#b2 = tf.Variable(tf.random_normal([4]))

#calculations
y_values = tf.add(tf.matmul(x,W),b)

y = tf.nn.softmax(y_values)

y_ = tf.placeholder(tf.float32,[None,4])

In [48]:
test = pd.read_csv('data_nn.csv',delimiter = ' ')
print (test)
time = test.ix[:,:1]
test = test.ix[:,1:9]

        Time  Ping Local  Ping LocalApp-CorpApp  Ping LocalApp-CorpDB  \
0       12:0       0.412                   68.3                  55.2   
1       12:1       0.474                   67.4                  54.9   
2       12:2       0.422                   66.7                  55.4   
3       12:3       0.586                   66.4                  55.3   
4       12:4       0.489                   74.4                  55.2   
5       12:5       0.383                   67.4                  56.5   
6       12:6       0.434                   66.7                  56.4   
7       12:7       0.304                   65.1                  56.9   
8       12:8       0.363                   66.1                  56.6   
9       12:9       0.398                   65.6                  56.1   
10     12:10       0.454                   65.5                  56.2   
11     12:11       0.405                   65.9                  56.6   
12     12:12       0.359                   66.4    

/home/aksharma/miniconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate_ix
  This is separate from the ipykernel package so we can avoid doing imports until


In [49]:
  inputX = test.as_matrix()
timeX = time.as_matrix()

In [50]:
with tf.Session() as sess:
    init = tf.initialize_all_variables()
    sess.run(init)
    result = sess.run(y,feed_dict={x:inputX})
rows = result.shape[0]

Instructions for updating:
Use `tf.global_variables_initializer` instead.


In [51]:
file = open('Time-Report.txt','w')
flag=0
for i in range(rows):
    if result[i][1]==1:
        if(flag!=1):
            file.write("At "+str(timeX[i])+"Remote App is down\n")
        flag=1
    elif result[i][2]==1:
        if(flag!=2):
            file.write("At "+str(timeX[i])+"Remote DB is down\n")
        flag=2
    elif result[i][3]==1:
        if(flag!=3):
            file.write("At "+str(timeX[i])+"Local DB is down\n")
        flag=3
    else:
        if(flag==1):
            file.write("At "+str(timeX[i])+"Remote App Back Up and Running\n\n")
        elif(flag==2):
            file.write("At "+str(timeX[i])+"Remote DB Back Up and Running\n\n")
        elif(flag==3):
            file.write("At "+str(timeX[i])+"Local DB Back Up and Running\n\n")
        flag=0
        
        
        
file.close()
        